# <font color = 'green'>스태킹 앙상블 p.278~ 

여러 개의 모델에 대한 예측값을 합한 후(스태킹 한 후), 이에 대한 예측을 최종 메타 모델이 수행하는 것

## ✔ 기본 스태킹 모델 

In [1]:
# 기본 스태킹 모델이 위스콘신 유방암 데이터 세트 적용
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
cancer_data = load_breast_cancer()

x_data = cancer_data.data
y_label = cancer_data.target

x_train,x_test, y_train, y_test = train_test_split(x_data, y_label, test_size = 0.2, random_state = 0)

### 머신러닝 알고리즘 클래스 생성

- ***개별모델*** : KNN, 램덤 포레스트. 결정트리. 에이다부스트
- ***최종 메타 모델*** : 로지스틱 회귀

In [3]:
# 개별 ML모델 생성
knn_clf = KNeighborsClassifier(n_neighbors = 4)
rf_clf = RandomForestClassifier(n_estimators = 100, random_state = 0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators = 100)

# 최종 메타 모델
# 스태킹으로 만들어진 데이터 세트 학습/예측 예정
lr_final = LogisticRegression(C = 10)

In [4]:
# 개별 모델 학습
knn_clf.fit(x_train,y_train)
rf_clf.fit(x_train,y_train)
dt_clf.fit(x_train,y_train)
ada_clf.fit(x_train,y_train)

AdaBoostClassifier(n_estimators=100)

In [5]:
# 개별 모델의 예측 데이터 세트를 반환하고 각 모델의 예측 정확도 살펴보기
knn_pred = knn_clf.predict(x_test)
rf_pred = rf_clf.predict(x_test)
dt_pred = dt_clf.predict(x_test)
ada_pred = ada_clf.predict(x_test)

print('knn 예측 정확도 : {0:.4f}'.format(accuracy_score(y_test, knn_pred)))
print('랜덤 포레스트 정확도 : {0:.4f}'.format(accuracy_score(y_test, rf_pred)))
print('결정 트리 정확도 : {0:.4f}'.format(accuracy_score(y_test, dt_pred)))
print('에이다부스트 정확도 : {0:.4f}'.format(accuracy_score(y_test, ada_pred)))

knn 예측 정확도 : 0.9211
랜덤 포레스트 정확도 : 0.9649
결정 트리 정확도 : 0.9123
에이다부스트 정확도 : 0.9561


In [6]:
# 개별 알고리즘으로 예측된 예측값을 칼럼 레벨로 옆으로 붙이기 -> 피처 값 생성
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)

# transpose를 이용하여 행과 열의 위치 교환. 칼럼 레벨로 각 알고리즘의 예측 결과를 피처로 만듦
pred = np.transpose(pred)
print(pred.shape)

(4, 114)
(114, 4)


In [7]:
# 최종 메타 모델에서 학습데이터로 사용
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

print('최종 메타 모델의 예측 정확도 : {0:.4f}'.format(accuracy_score(y_test,final)))


최종 메타 모델의 예측 정확도 : 0.9737


## ✔CV 세트 기반의 스태킹


- 과적합 개선을 위한 스태킹
- 최종 메타 모델을 위한 데이터 세트 생성 시 **교차 검증** 기반으로 예측된 결과 데이터 세트를 이용함

### 스탭 1 

- 교차 검층을 통해 각 모델별 원본 학습/테스트 데이터를 예측한 결과 값을 기반으로 메타 모델을 위한 학습용/테스트용 데이터를 생성한다.


In [17]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터 생성
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # KFold 생성
    kf = KFold(n_splits=n_folds, shuffle=False)
    
    # 메타 모델이 사용할 학습 데이터 세트 반환을 위해 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ) ) # 최종 메타 모델이 사용할 학습데이터
    test_pred = np.zeros((X_test_n.shape[0],n_folds)) # 폴드 세트에서 모델들의 예측 데이터
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        # 입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 세트 추출
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n[train_index] 
        y_tr = y_train_n[train_index] 
        X_te = X_train_n[valid_index]  
        
        # 폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        
        # 폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        
        # 입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean
    

In [18]:
# 여러 개의 분류 모델별로 stack_basE_model() 함수 실행
knn_train, knn_test = get_stacking_base_datasets(knn_clf, x_train, y_train, x_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, x_train, y_train, x_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, x_train, y_train, x_test,  7)    
ada_train, ada_test = get_stacking_base_datasets(ada_clf, x_train, y_train, x_test, 7)

KNeighborsClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
RandomForestClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
DecisionTreeClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
AdaBoostClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 


### 스탭 2

- 1에서 생성한 학습용/테스트용 데이터를 모두 스태킹 형태로 합쳐서 메타 모델이 학습할 최종 학습용/테스트용 데이터 세트를 생성한다.

- 메타모델은 이렇게 생성된 학습 데이터 세트와 원본 학습 데이터의 레이블 데이터를 기반으로 학습한 뒤, 최종적으로 생성된 테스트 데이터 세트를 예측하고, 원본 데이터의 레이블 데이터를 기반으로 평가한다.

In [20]:
# 각 모델들이 반환한 학습 데이터와 테스트 데이터 합치기 -> concatenate 활용
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1) # 학습용
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1) # 테스트용
print('원본 학습 피처 데이터 Shape:',x_train.shape, '원본 테스트 피처 Shape:',x_test.shape)
print('스태킹 학습 피처 데이터 Shape:', Stack_final_X_train.shape,
      '스태킹 테스트 피처 데이터 Shape:',Stack_final_X_test.shape)

원본 학습 피처 데이터 Shape: (455, 30) 원본 테스트 피처 Shape: (114, 30)
스태킹 학습 피처 데이터 Shape: (455, 4) 스태킹 테스트 피처 데이터 Shape: (114, 4)


In [21]:
# 최종 메타 모델인 로지스틱 회귀를 학습시키기
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test, stack_final)))

최종 메타 모델의 예측 정확도: 0.9737
